In [21]:
import pandas as pd
import math

In [22]:
def _isLetter(letter):
    # 0 a 47
    # 58 a 63
    # 91 a 96
    # 123 a 126
    # 8200,8230
    
    codeLetter = ord(letter)

    return not ( 0 <= codeLetter <= 57 
    or 58 <= codeLetter <= 63 
    or 91 <= codeLetter <= 96 
    or 123 <= codeLetter <= 126 
    or 8200 <= codeLetter <= 8230)


In [23]:
ord("9")

57

In [24]:
chr(98)

'b'

In [25]:
def cleaningWord(word):

    i = 0
    n = len(word)
    
    while i < n and not _isLetter(word[i]):
        i += 1

    j = n
    while j > 0 and not _isLetter(word[j-1]):
        j -= 1
    
 
    word = word[i:j]
    
    return word.lower()

In [26]:
def tf(palavra, doc):
    # palavra : total de palavras que aparceu no doc
    # doc : numero total de palavras do doc
    return palavra/doc

In [27]:
def n_containing(palavra, docs):
    # retorna o numero de documento que tem a palvra
    pass

In [28]:
def idf(qntWords, docs):
    # Palavra: total de documentos que tem a palavra
    # docs: total de documentos
    h= docs/(qntWords if qntWords > 0 else 1)
    return math.log2(h)
  

In [29]:
def tfidf(palavra, doc,qntWords, docs):
    return tf(palavra, doc) * idf(qntWords, docs)

In [30]:
def loadStopWords():
    file_ = open("../data/stopwords_br.txt")
    wordsStop = []
    linhas = file_.readlines()

    for line in linhas:
        wordsStop.append(cleaningWord(line))

    return wordsStop

In [31]:
def validate(word,stopW):
    return not cleaningWord(word) in stopW


In [32]:
def getTotWords(file,stopWords,doc, start, end):
    wordsInDoc = {}
    totWord = 0

    for idxLine in range(start, end+1):
        
        wos = [ cleaningWord(x) for x in file[idxLine].split(" ") if validate(x,stopWords)]

        for wo in wos:
            totWord += 1
            if not wo in wordsInDoc.keys():
                wordsInDoc[wo] = 1
            else:
                wordsInDoc[wo] += 1
    return wordsInDoc, totWord

In [33]:
def mappingPages(file):
    totLines = len(file)

    pages = {}
    startPage = 0
    endPage = 0
    ultPage = 1
    i = 1

    while i < totLines:

        subStr = file[i][:2]

        if (subStr == "PA"):
            endPage = i-1
            pages[ultPage] = {
                "start":startPage,
                "end": endPage
            }

            startPage = i
            ultPage += 1

        i+=1


    endPage = i-1
    pages[ultPage] = {
        "start":startPage,
        "end": endPage
    }

    return pages

    

In [34]:
def preprareTable(dicResult, totDocs, totWords):
    
    words = []
    recurrences = []
    pontuation = []
    pagesOc = []

    for wd in dicResult:
        words.append(wd)
        recurrences.append(dicResult[wd]["totRecorences"])

        pont = 0
        stringTmp = ""
        for page in dicResult[wd]["pages"]:
            tmp = tfidf(dicResult[wd]["pages"][page]["recorrences"], dicResult[wd]["pages"][page]["totWord"], dicResult[wd]["totPages"], totDocs)
            pont += tmp
            stringTmp = f'{stringTmp} -> {page}: {dicResult[wd]["pages"][page]["totWord"]}'

        toP = dicResult[wd]["totPages"] if dicResult[wd]["totPages"] > 0 else 1

        pont = pont/toP

        pontuation.append( pont )
        pagesOc.append( stringTmp )

    
    d = {'palavra': words, 'rec': recurrences, "pont": pontuation}
        
    return pd.DataFrame(data=d)


In [35]:
# file_ = open("../data/Paralelismo.base")
# ffile_ = open("../data/Guarani.base")
file_ = open("../data/Aventuras.base")
lines = file_.readlines()

pages = mappingPages(lines)
print(f"Pages: {len(pages)}")

Pages: 264


In [36]:
words = {}
wordsResul = {}

wordCont = 0

stopWords = loadStopWords()

for page in pages:

    start = pages[page]["start"]
    end = pages[page]["end"]

    words, totWord = getTotWords(lines,stopWords, page, start, end)
    for wd in words:
        if len(wd) == 0:
            continue

        wordCont += 1
            
        if not wd in wordsResul.keys():
            wordsResul[wd] = {
                "pages":{
                    page:{
                        "recorrences": words[wd],
                        "totWord": totWord
                    }
                },
                "totPages": 1,
                "totRecorences": words[wd]
            }
        else:
            wordsResul[wd]["pages"][page] = {
                "recorrences": words[wd],
                "totWord": totWord
            }
            wordsResul[wd]["totPages"] += 1
            wordsResul[wd]["totRecorences"] += words[wd]

In [37]:
table = preprareTable(wordsResul, len(pages), wordCont)

# table = table.loc[table["rec"] > 1]
# table = table.loc[table["rec"] <= 5]
table = table.sort_values(by="pont", ascending=False)
print(table)

table.to_csv("Paralelismo.csv")


        palavra  rec      pont
4816      sonho    5  0.132746
8122   exéquias    5  0.132746
6678      vinde    5  0.129748
10705    cótica    5  0.127689
11255  compadre   11  0.125992
...         ...  ...       ...
485       disse  151  0.005096
431        pois  173  0.005076
1058      mesmo  167  0.005022
567          só  235  0.004285
0            pa  264  0.000000

[11581 rows x 3 columns]


In [5]:
def lowerCase(string):

    size = len(string)

    for i in range(size):

        if ord(string[i]) >= 65 and ord(string[i]) <= 90:
            string[i] = chr(ord(string[i]) + 32)
        elif(ord(string[i]) >= 193 and ord(string[i]) <= 214):
            string[i] = chr(ord(string[i]) + 32)
        
    
    return string

In [6]:
lowerCase("VISÃo")

TypeError: 'str' object does not support item assignment

In [1]:
name = "Natanael"

dec = {
    "name":name
}

dec["name"]

'Natanael'

In [2]:
name = "Porra"
dec["name"]

'Natanael'

In [3]:
print(name)

Porra
